In [ ]:
import openai
import pandas as pd
import numpy as np
import random
random.seed(2022)

openai.api_key = "KEY"

df = pd.read_csv("../../data/analysis_of_prior_work/aopw_bloom_response_by_study/....csv")
df

In [34]:
asked_question_cheat_sheet = {}

In [35]:
def maybe_ask_gpt3(sentence, prefix_prompt):
    sentence = sentence.strip()
    if sentence in asked_question_cheat_sheet:
        return asked_question_cheat_sheet[sentence], False #Skip asking gpt3 again if the sentence existes in the cheat sheet
    else:
        prompt = prefix_prompt.strip() + "\n\n" + f"Sentence: {sentence}\n"  + "Answer:"
        answer = (
            openai.Completion.create(
                engine="text-curie-001", 
                prompt=prompt,
                temperature = 0,
                top_p=0.1,
                max_tokens=5)).choices[0].text.strip()
        asked_question_cheat_sheet[sentence] = answer
        return answer, True

In [36]:
def truncate_response(sentence): #truncate the original responses
    cutoff_pos = sentence.find(".")
    cutoff_pos = cutoff_pos + 1 if cutoff_pos >= 0 else None
    return sentence[:cutoff_pos].split("\n")[0].strip()

In [ ]:
for col in df.columns:
    if col.startswith("bloom_response_question_"):
        print(col)
        print(df[col].apply(truncate_response).values)
        df[f"{col}_truncated"] = df[col].apply(truncate_response).values

In [ ]:
df = df.fillna("")

categorization_rating_column = []
charged_by_gpt3_cnt = 0

for row in range(100):
    prefix_prompt = """


    """
    categorization_rating, is_charged_by_gpt3 = maybe_ask_gpt3(df["bloom_response_question_1_1_truncated"][row], prefix_prompt)
    if is_charged_by_gpt3:
        charged_by_gpt3_cnt += 1

    print(df["bloom_response_question_1_1_truncated"][row])
    print(categorization_rating)
    print("------------------------")

    categorization_rating_column.append(categorization_rating)


df["categorization_rating"] = categorization_rating_column

# Optional: 
# print(f"Charged by GPT-3 for {charged_by_gpt3_cnt} / {len(categorization_rating_column)} times")


In [23]:
df.to_csv("../../data/analysis_of_prior_work/aopw_bloom_response_by_study_answer_retrieved/....csv", index=False)